# qwen3-30b-a3b-instruct-2507 and Gold standard

Evaluative: Cohen's Kappa: 0.3669510624597553


Skipping Evaluative: Only one unique value ([1]) after filtering for Evaluative=1.

Krippendorff's Alpha for Affect: 0.399

Krippendorff's Alpha for Judgement: 0.228

Krippendorff's Alpha for Appreciation: 0.334

Krippendorff's Alpha for Ambiguous: -0.

---

# llama-3.3-70b-instruct.csv and Gold standard

Evaluative: Cohen's Kappa: 0.34060866892099595

Skipping Evaluative: Only one unique value ([1]) after filtering for Evaluative=1.

Krippendorff's Alpha for Affect: 0.471

Krippendorff's Alpha for Judgement: 0.230

Krippendorff's Alpha for Appreciation: 0.330

Krippendorff's Alpha for Ambiguous: -0.071

---


# mistral-large-instruct and Gold standard

Evaluative: Cohen's Kappa: 0.35751613894866274
Skipping Evaluative: Only one unique value ([1]) after filtering for Evaluative=1.

Krippendorff's Alpha for Affect: 0.367

Krippendorff's Alpha for Judgement: 0.326

Krippendorff's Alpha for Appreciation: 0.295

Krippendorff's Alpha for Ambiguous: -0.066




In [44]:

import pandas as pd
import numpy as np

#df = pd.read_excel("non-arg-phatic.xlsx")

df = pd.read_csv("/content/qwen3-30b-a3b-instruct-2507.csv")
df2 = pd.read_excel("/content/Gold_standard_Mirela.xlsx")


In [45]:
df.columns

Index(['Unnamed: 0', 'Text_ID', 'Sentence_ID', 'Sentence', 'Evaluative',
       'Affect', 'Judgement', 'Appreciation', 'Ambiguous', 'binary',
       'binary_justification', 'binary_evidence_span', 'multiclass_label',
       'multiclass_probability_affect', 'multiclass_probability_judgment',
       'multiclass_probability_appreciation', 'multiclass_spans',
       'multiclass_explanation', 'binary_raw_output', 'multiclass_raw_output',
       'Source_File'],
      dtype='object')

In [47]:
df = df.drop(['Unnamed: 0', 'Sentence', 'Evaluative','multiclass_label',
       'Affect', 'Judgement', 'Appreciation', 'Ambiguous', 'Source_File','multiclass_explanation','binary_justification','binary_raw_output', 'multiclass_raw_output','multiclass_spans','multiclass_spans','binary_evidence_span'],axis = "columns")

In [49]:
# Columns to process
prob_cols = [
    "multiclass_probability_affect",
    "multiclass_probability_judgment",
    "multiclass_probability_appreciation"
]

# 1. Convert NaN (and string 'nan') to 0
df[prob_cols] = df[prob_cols].replace(["nan", "NaN", "NAN"], np.nan)
df[prob_cols] = df[prob_cols].fillna(0)

# 2. Convert to numeric and threshold >=0.5 → 1 else → 0
df[prob_cols] = df[prob_cols].apply(pd.to_numeric, errors="coerce").fillna(0)
df[prob_cols] = (df[prob_cols] >= 0.5).astype(int)

# 3. Create Ambiguous = 1 if more than 2 of the 3 values are 1
df["Ambiguous"] = (df[prob_cols].sum(axis=1) > 2).astype(int)

# 4. Rename columns
df = df.rename(columns={
    "multiclass_probability_affect": "Affect",
    "multiclass_probability_judgment": "Judgement",
    "multiclass_probability_appreciation": "Appreciation",
    "binary": "Evaluative"
})



In [50]:

# CHECK WHAT OTHER LABELS ARE TRUE PER ANNOTATOR IF EVALUATIVE = 0

df.columns

df = df.fillna(0)
#df2 = df2.drop(index=5)

df['Evaluative'] = df['Evaluative'].astype(int)
df['Affect'] = df['Affect'].astype(int)
df['Judgement'] = df['Judgement'].astype(int)
df['Appreciation'] = df['Appreciation'].astype(int)
df['Ambiguous'] = df['Ambiguous'].astype(int)
#df['Uncertain'] = df['Uncertain'].astype(int)


In [51]:
df2 = df2.fillna(0)

df2['Evaluative'] = df2['Evaluative'].astype(int)
df2['Affect'] = df2['Affect'].astype(int)
df2['Judgement'] = df2['Judgement'].astype(int)
df2['Appreciation'] = df2['Appreciation'].astype(int)
df2['Ambiguous'] = df2['Ambiguous'].astype(int)
#df2['Uncertain'] = df2['Uncertain'].astype(int)

In [92]:
df = df.drop_duplicates(subset=["Text_ID", "Sentence_ID"])
df = df.set_index(["Text_ID", "Sentence_ID"])
df2_idx = df2.set_index(["Text_ID", "Sentence_ID"]).index

df = df.loc[df2_idx].reset_index()
df

,Text_ID,Sentence_ID,Evaluative,Affect,Judgement,Appreciation,Ambiguous
0,Art5.,S6,0,0,0,0,0
1,Art5.,S120,1,1,0,0,0
2,Art5.,S51,1,1,0,0,0
3,Art5.,S131,1,1,0,0,0
4,Art5.,S130,1,1,0,0,0
...,...,...,...,...,...,...,...
435,Tech21.,S105,1,0,0,1,0
436,Tech21.,S30,0,0,0,0,0
437,Tech21.,S68,0,0,0,0,0
438,Tech21.,S66,0,0,0,0,0


In [90]:
df2

,Text_ID,Sentence_ID,Sentence,Evaluative,Affect,Judgement,Appreciation,Ambiguous
0,Art5.,S6,"Yes, eight kids in my family.",1,0,0,1,0
1,Art5.,S120,"When the idea hit me of how to do this, I got ...",1,1,0,1,1
2,Art5.,S51,"When she told me this, I fell to my knees.",1,1,0,0,0
3,Art5.,S131,"We're going to lose our health, people we love.",1,1,0,0,0
4,Art5.,S130,We're all going to have big losses in our live...,1,1,0,0,0
...,...,...,...,...,...,...,...,...
435,Tech21.,S105,We are less connected to it.,1,1,1,0,1
436,Tech21.,S30,Was the Internet actually a place that you cou...,0,0,0,0,0
437,Tech21.,S68,"Three years ago, when I started thinking about...",0,0,0,0,0
438,Tech21.,S66,"This is in Halifax, a cable that stretches fro...",0,0,0,0,0


## Analysis of Evaulative column = 0 and other columns = 1

In [93]:

df[(df['Evaluative'].isna()) | (df['Evaluative'] == "")]

df2[(df2['Evaluative'].isna()) | (df2['Evaluative'] == "")]

,Text_ID,Sentence_ID,Sentence,Evaluative,Affect,Judgement,Appreciation,Ambiguous


In [94]:
cols_annotator_1  = ['Evaluative', 'Affect', 'Judgement', 'Appreciation', 'Ambiguous']
cols_annotator_2  = ['Evaluative', 'Affect', 'Judgement', 'Appreciation', 'Ambiguous']

In [95]:
# Filter rows where Evaluative == 0 for annotator 1
df_0_eval_1 = df[df['Evaluative'] == 0]

In [96]:
# Count how many times each of the other columns == 1 in those rows
counts_1 = df_0_eval_1[cols_annotator_1].apply(lambda col: (col == 1).sum())
counts_1

,0
Evaluative,0
Affect,0
Judgement,0
Appreciation,0
Ambiguous,0


In [97]:
# Repeat for annotator 2
df_0_eval_2 = df[df['Evaluative'] == 0]
counts_2 = df_0_eval_2[cols_annotator_2].apply(lambda col: (col == 1).sum())

In [98]:
print("Annotator 1 - counts where Evaluative=0 and other columns=1:\n", counts_1)
print("Annotator 2 - counts where Evaluative=0 and other columns=1:\n", counts_2)

Annotator 1 - counts where Evaluative=0 and other columns=1:
 Evaluative      0
Affect          0
Judgement       0
Appreciation    0
Ambiguous       0
dtype: int64
Annotator 2 - counts where Evaluative=0 and other columns=1:
 Evaluative      0
Affect          0
Judgement       0
Appreciation    0
Ambiguous       0
dtype: int64


## Analysis of Evaulative column = 1

In [99]:
# Filter rows where Evaluative == 1 for annotator 1
df_0_eval_1 = df[df['Evaluative'] == 1]

In [100]:
# Count how many times each of the other columns == 1 in those rows
counts_1 = df_0_eval_1[cols_annotator_1].apply(lambda col: (col == 1).sum())
counts_1

,0
Evaluative,193
Affect,51
Judgement,45
Appreciation,122
Ambiguous,2


In [101]:
# Repeat for annotator 2
df_0_eval_2 = df2[df2['Evaluative'] == 1]
counts_2 = df_0_eval_2[cols_annotator_2].apply(lambda col: (col == 1).sum())
counts_2

,0
Evaluative,268
Affect,61
Judgement,127
Appreciation,107
Ambiguous,27


In [102]:
print("Annotator 1 - counts where Evaluative=1 and other columns=1:\n", counts_1)
print("Annotator 2 - counts where Evaluative=1 and other columns=1:\n", counts_2)

Annotator 1 - counts where Evaluative=1 and other columns=1:
 Evaluative      193
Affect           51
Judgement        45
Appreciation    122
Ambiguous         2
dtype: int64
Annotator 2 - counts where Evaluative=1 and other columns=1:
 Evaluative      268
Affect           61
Judgement       127
Appreciation    107
Ambiguous        27
dtype: int64


## Berechnung: Cohens_Kappa

In [105]:
!pip install krippendorff

In [103]:
from sklearn.metrics import cohen_kappa_score


In [ ]:
"""# Filter out the placeholder values before calculating the kappa score
df_anno1_filtered = df_anno1[df_anno1["Evaluative"] != -1]
df2_anno2_filtered = df2_anno2[df2_anno2["Evaluative"] != -1]

# Ensure both dataframes have the same indices after filtering
common_indices = df_anno1_filtered.index.intersection(df2_anno2_filtered.index)
df_anno1_aligned = df_anno1_filtered.loc[common_indices]
df2_anno2_aligned = df2_anno2_filtered.loc[common_indices]"""

In [104]:
# Assuming your columns are named "mobilisierung" and "informativ" (adjust as necessary)
kappa_score = cohen_kappa_score(df["Evaluative"], df2["Evaluative"])
print(f"Cohen's Kappa: {kappa_score}")

Cohen's Kappa: 0.3669510624597553


In [ ]:
#Kappa Kuck und Knierim: Cohen's Kappa: 0.6156981445684476
# Kappa Knierim und GPT4: Cohen's Kappa: 0.35816326807273824
# Kappa Kuck und GPT4: Cohen's Kappa: 0.38653552825615034

In [ ]:
#filter the empty string-lines to calculate agreement on whether the sentence is evaluative
"""df[df['Evaluative'] == 1]
df2[df2['Evaluative'] == 1]
"""

In [106]:
import krippendorff
import numpy as np

def calculate_krippendorffs_alpha(df_list, columns, evaluative_col='Evaluative'):
    """
    Calculates Krippendorff's Alpha for given columns, considering only rows
    where evaluative_col == 1. Assumes missing data is marked as -1.

    Parameters:
        df_list (list of pd.DataFrame): List of annotator DataFrames (one per annotator).
        columns (list): List of column names to calculate alpha on.
        evaluative_col (str): Column name indicating 'true' evaluative lines.

    Returns:
        dict: column_name -> Krippendorff's Alpha score
    """
    alphas = {}

    # Combine DataFrames on index (assumes they are aligned)
    # Create a matrix of shape (num_annotators, num_items)
    for col in columns:
        # Filter rows where evaluative_col == 1 for all annotators
        mask = np.array([df[evaluative_col] == 1 for df in df_list]).all(axis=0)

        # Collect data from all annotators for this column, filtered by mask
        data = []
        for df in df_list:
            # Use -1 as missing, krippendorff expects np.nan for missing, so convert
            col_data = df.loc[mask, col].replace(-1, np.nan).to_numpy()
            data.append(col_data)

        # Convert to numpy array of shape (num_annotators, num_items)
        data = np.array(data)

        # Check if there is more than one unique value in the data for this column
        unique_values = np.unique(data[~np.isnan(data)])
        if len(unique_values) <= 1:
            print(f"Skipping {col}: Only one unique value ({unique_values}) after filtering for Evaluative=1.")
            continue

        # Calculate Krippendorff's Alpha (nominal metric)
        alpha = krippendorff.alpha(reliability_data=data, level_of_measurement='nominal')
        alphas[col] = alpha

    return alphas

df_list = [df, df2]
cols_to_check = ['Evaluative','Affect', 'Judgement', 'Appreciation', 'Ambiguous']

alphas = calculate_krippendorffs_alpha(df_list, cols_to_check)
for col, alpha in alphas.items():
    print(f"Krippendorff's Alpha for {col}: {alpha:.3f}")



Skipping Evaluative: Only one unique value ([1]) after filtering for Evaluative=1.
Krippendorff's Alpha for Affect: 0.399
Krippendorff's Alpha for Judgement: 0.228
Krippendorff's Alpha for Appreciation: 0.334
Krippendorff's Alpha for Ambiguous: -0.064


In [ ]:

"""def save_agreement_disagreement_csvs_allcols(df, df2, columns_to_check, output_agree='agree.csv', output_disagree='disagree.csv'):

    Save two CSVs with all columns:
    - rows where annotators agree in ALL specified columns_to_check
    - rows where annotators disagree in ANY of those columns

    Parameters:
        df1, df2 (pd.DataFrame): Annotator DataFrames, aligned by index
        columns_to_check (list): Columns to check agreement on
        output_agree (str): Filename for agreed sentences CSV
        output_disagree (str): Filename for disagreed sentences CSV

    # Agreement mask for all specified columns
    agreement_mask = pd.Series(True, index=df.index)
    for col in columns_to_check:
        agreement_mask &= (df[col] == df2[col])

    # Select all columns from df1 (or df2, they should have same columns)
    agreed_rows = df.loc[agreement_mask].copy()
    disagreed_rows = df.loc[~agreement_mask].copy()

    agreed_rows.to_csv(output_agree, index=False)
    disagreed_rows.to_csv(output_disagree, index=False)

    print(f"Saved {len(agreed_rows)} agreed rows to {output_agree}")
    print(f"Saved {len(disagreed_rows)} disagreed rows to {output_disagree}")

cols_to_check = ['Affect', 'Judgement', 'Appreciation']

save_agreement_disagreement_csvs_allcols(df, df2, cols_to_check,
                                         output_agree='agreed_full_Bus11.csv',
                                         output_disagree='disagreed_full_Buss11.csv')"
"""


# llama-3.3-70b-instruct.csv and Gold standard

Evaluative: Cohen's Kappa: 0.34060866892099595

Skipping Evaluative: Only one unique value ([1]) after filtering for Evaluative=1.

Krippendorff's Alpha for Affect: 0.471

Krippendorff's Alpha for Judgement: 0.230

Krippendorff's Alpha for Appreciation: 0.330

Krippendorff's Alpha for Ambiguous: -0.071

In [107]:

import pandas as pd
import numpy as np

#df = pd.read_excel("non-arg-phatic.xlsx")

df = pd.read_csv("/content/llama-3.3-70b-instruct.csv")
df2 = pd.read_excel("/content/Gold_standard_Mirela.xlsx")


In [108]:
df.columns

Index(['Unnamed: 0', 'Text_ID', 'Sentence_ID', 'Sentence', 'Evaluative',
       'Affect', 'Judgement', 'Appreciation', 'Ambiguous', 'binary',
       'binary_justification', 'binary_evidence_span', 'multiclass_label',
       'multiclass_probability_affect', 'multiclass_probability_judgment',
       'multiclass_probability_appreciation', 'multiclass_spans',
       'multiclass_explanation', 'binary_raw_output', 'multiclass_raw_output',
       'Source_File'],
      dtype='object')

In [109]:
df = df.drop(['Unnamed: 0', 'Sentence', 'Evaluative','multiclass_label',
       'Affect', 'Judgement', 'Appreciation', 'Ambiguous', 'Source_File','multiclass_explanation','binary_justification','binary_raw_output', 'multiclass_raw_output','multiclass_spans','multiclass_spans','binary_evidence_span'],axis = "columns")

In [110]:
# Columns to process
prob_cols = [
    "multiclass_probability_affect",
    "multiclass_probability_judgment",
    "multiclass_probability_appreciation"
]

# 1. Convert NaN (and string 'nan') to 0
df[prob_cols] = df[prob_cols].replace(["nan", "NaN", "NAN"], np.nan)
df[prob_cols] = df[prob_cols].fillna(0)

# 2. Convert to numeric and threshold >=0.5 → 1 else → 0
df[prob_cols] = df[prob_cols].apply(pd.to_numeric, errors="coerce").fillna(0)
df[prob_cols] = (df[prob_cols] >= 0.5).astype(int)

# 3. Create Ambiguous = 1 if more than 2 of the 3 values are 1
df["Ambiguous"] = (df[prob_cols].sum(axis=1) > 2).astype(int)

# 4. Rename columns
df = df.rename(columns={
    "multiclass_probability_affect": "Affect",
    "multiclass_probability_judgment": "Judgement",
    "multiclass_probability_appreciation": "Appreciation",
    "binary": "Evaluative"
})



In [111]:

# CHECK WHAT OTHER LABELS ARE TRUE PER ANNOTATOR IF EVALUATIVE = 0

df.columns

df = df.fillna(0)
#df2 = df2.drop(index=5)

df['Evaluative'] = df['Evaluative'].astype(int)
df['Affect'] = df['Affect'].astype(int)
df['Judgement'] = df['Judgement'].astype(int)
df['Appreciation'] = df['Appreciation'].astype(int)
df['Ambiguous'] = df['Ambiguous'].astype(int)
#df['Uncertain'] = df['Uncertain'].astype(int)


In [112]:
df2 = df2.fillna(0)

df2['Evaluative'] = df2['Evaluative'].astype(int)
df2['Affect'] = df2['Affect'].astype(int)
df2['Judgement'] = df2['Judgement'].astype(int)
df2['Appreciation'] = df2['Appreciation'].astype(int)
df2['Ambiguous'] = df2['Ambiguous'].astype(int)
#df2['Uncertain'] = df2['Uncertain'].astype(int)

In [113]:
df = df.drop_duplicates(subset=["Text_ID", "Sentence_ID"])
df = df.set_index(["Text_ID", "Sentence_ID"])
df2_idx = df2.set_index(["Text_ID", "Sentence_ID"]).index

df = df.loc[df2_idx].reset_index()
df

,Text_ID,Sentence_ID,Evaluative,Affect,Judgement,Appreciation,Ambiguous
0,Art5.,S6,0,0,0,0,0
1,Art5.,S120,1,1,0,0,0
2,Art5.,S51,1,1,0,0,0
3,Art5.,S131,1,1,0,0,0
4,Art5.,S130,1,1,0,0,0
...,...,...,...,...,...,...,...
435,Tech21.,S105,1,0,0,1,0
436,Tech21.,S30,0,0,0,0,0
437,Tech21.,S68,0,0,0,0,0
438,Tech21.,S66,0,0,0,0,0


In [114]:
df2

,Text_ID,Sentence_ID,Sentence,Evaluative,Affect,Judgement,Appreciation,Ambiguous
0,Art5.,S6,"Yes, eight kids in my family.",1,0,0,1,0
1,Art5.,S120,"When the idea hit me of how to do this, I got ...",1,1,0,1,1
2,Art5.,S51,"When she told me this, I fell to my knees.",1,1,0,0,0
3,Art5.,S131,"We're going to lose our health, people we love.",1,1,0,0,0
4,Art5.,S130,We're all going to have big losses in our live...,1,1,0,0,0
...,...,...,...,...,...,...,...,...
435,Tech21.,S105,We are less connected to it.,1,1,1,0,1
436,Tech21.,S30,Was the Internet actually a place that you cou...,0,0,0,0,0
437,Tech21.,S68,"Three years ago, when I started thinking about...",0,0,0,0,0
438,Tech21.,S66,"This is in Halifax, a cable that stretches fro...",0,0,0,0,0


## Analysis of Evaulative column = 0 and other columns = 1

In [115]:

df[(df['Evaluative'].isna()) | (df['Evaluative'] == "")]

df2[(df2['Evaluative'].isna()) | (df2['Evaluative'] == "")]

,Text_ID,Sentence_ID,Sentence,Evaluative,Affect,Judgement,Appreciation,Ambiguous


In [116]:
cols_annotator_1  = ['Evaluative', 'Affect', 'Judgement', 'Appreciation', 'Ambiguous']
cols_annotator_2  = ['Evaluative', 'Affect', 'Judgement', 'Appreciation', 'Ambiguous']

In [117]:
# Filter rows where Evaluative == 0 for annotator 1
df_0_eval_1 = df[df['Evaluative'] == 0]

In [118]:
# Count how many times each of the other columns == 1 in those rows
counts_1 = df_0_eval_1[cols_annotator_1].apply(lambda col: (col == 1).sum())
counts_1

,0
Evaluative,0
Affect,0
Judgement,0
Appreciation,0
Ambiguous,0


In [119]:
# Repeat for annotator 2
df_0_eval_2 = df[df['Evaluative'] == 0]
counts_2 = df_0_eval_2[cols_annotator_2].apply(lambda col: (col == 1).sum())

In [120]:
print("Annotator 1 - counts where Evaluative=0 and other columns=1:\n", counts_1)
print("Annotator 2 - counts where Evaluative=0 and other columns=1:\n", counts_2)

Annotator 1 - counts where Evaluative=0 and other columns=1:
 Evaluative      0
Affect          0
Judgement       0
Appreciation    0
Ambiguous       0
dtype: int64
Annotator 2 - counts where Evaluative=0 and other columns=1:
 Evaluative      0
Affect          0
Judgement       0
Appreciation    0
Ambiguous       0
dtype: int64


## Analysis of Evaulative column = 1

In [121]:
# Filter rows where Evaluative == 1 for annotator 1
df_0_eval_1 = df[df['Evaluative'] == 1]

In [122]:
# Count how many times each of the other columns == 1 in those rows
counts_1 = df_0_eval_1[cols_annotator_1].apply(lambda col: (col == 1).sum())
counts_1

,0
Evaluative,144
Affect,30
Judgement,25
Appreciation,89
Ambiguous,0


In [123]:
# Repeat for annotator 2
df_0_eval_2 = df2[df2['Evaluative'] == 1]
counts_2 = df_0_eval_2[cols_annotator_2].apply(lambda col: (col == 1).sum())
counts_2

,0
Evaluative,268
Affect,61
Judgement,127
Appreciation,107
Ambiguous,27


In [124]:
print("Annotator 1 - counts where Evaluative=1 and other columns=1:\n", counts_1)
print("Annotator 2 - counts where Evaluative=1 and other columns=1:\n", counts_2)

Annotator 1 - counts where Evaluative=1 and other columns=1:
 Evaluative      144
Affect           30
Judgement        25
Appreciation     89
Ambiguous         0
dtype: int64
Annotator 2 - counts where Evaluative=1 and other columns=1:
 Evaluative      268
Affect           61
Judgement       127
Appreciation    107
Ambiguous        27
dtype: int64


## Berechnung: Cohens_Kappa

In [125]:
!pip install krippendorff

In [126]:
from sklearn.metrics import cohen_kappa_score


In [127]:
"""# Filter out the placeholder values before calculating the kappa score
df_anno1_filtered = df_anno1[df_anno1["Evaluative"] != -1]
df2_anno2_filtered = df2_anno2[df2_anno2["Evaluative"] != -1]

# Ensure both dataframes have the same indices after filtering
common_indices = df_anno1_filtered.index.intersection(df2_anno2_filtered.index)
df_anno1_aligned = df_anno1_filtered.loc[common_indices]
df2_anno2_aligned = df2_anno2_filtered.loc[common_indices]"""

'# Filter out the placeholder values before calculating the kappa score\ndf_anno1_filtered = df_anno1[df_anno1["Evaluative"] != -1]\ndf2_anno2_filtered = df2_anno2[df2_anno2["Evaluative"] != -1]\n\n# Ensure both dataframes have the same indices after filtering\ncommon_indices = df_anno1_filtered.index.intersection(df2_anno2_filtered.index)\ndf_anno1_aligned = df_anno1_filtered.loc[common_indices]\ndf2_anno2_aligned = df2_anno2_filtered.loc[common_indices]'

In [128]:
# Assuming your columns are named "mobilisierung" and "informativ" (adjust as necessary)
kappa_score = cohen_kappa_score(df["Evaluative"], df2["Evaluative"])
print(f"Cohen's Kappa: {kappa_score}")

Cohen's Kappa: 0.34060866892099595


In [129]:
#Kappa Kuck und Knierim: Cohen's Kappa: 0.6156981445684476
# Kappa Knierim und GPT4: Cohen's Kappa: 0.35816326807273824
# Kappa Kuck und GPT4: Cohen's Kappa: 0.38653552825615034

In [130]:
#filter the empty string-lines to calculate agreement on whether the sentence is evaluative
"""df[df['Evaluative'] == 1]
df2[df2['Evaluative'] == 1]
"""

"df[df['Evaluative'] == 1]\ndf2[df2['Evaluative'] == 1]\n"

In [131]:
import krippendorff
import numpy as np

def calculate_krippendorffs_alpha(df_list, columns, evaluative_col='Evaluative'):
    """
    Calculates Krippendorff's Alpha for given columns, considering only rows
    where evaluative_col == 1. Assumes missing data is marked as -1.

    Parameters:
        df_list (list of pd.DataFrame): List of annotator DataFrames (one per annotator).
        columns (list): List of column names to calculate alpha on.
        evaluative_col (str): Column name indicating 'true' evaluative lines.

    Returns:
        dict: column_name -> Krippendorff's Alpha score
    """
    alphas = {}

    # Combine DataFrames on index (assumes they are aligned)
    # Create a matrix of shape (num_annotators, num_items)
    for col in columns:
        # Filter rows where evaluative_col == 1 for all annotators
        mask = np.array([df[evaluative_col] == 1 for df in df_list]).all(axis=0)

        # Collect data from all annotators for this column, filtered by mask
        data = []
        for df in df_list:
            # Use -1 as missing, krippendorff expects np.nan for missing, so convert
            col_data = df.loc[mask, col].replace(-1, np.nan).to_numpy()
            data.append(col_data)

        # Convert to numpy array of shape (num_annotators, num_items)
        data = np.array(data)

        # Check if there is more than one unique value in the data for this column
        unique_values = np.unique(data[~np.isnan(data)])
        if len(unique_values) <= 1:
            print(f"Skipping {col}: Only one unique value ({unique_values}) after filtering for Evaluative=1.")
            continue

        # Calculate Krippendorff's Alpha (nominal metric)
        alpha = krippendorff.alpha(reliability_data=data, level_of_measurement='nominal')
        alphas[col] = alpha

    return alphas

df_list = [df, df2]
cols_to_check = ['Evaluative','Affect', 'Judgement', 'Appreciation', 'Ambiguous']

alphas = calculate_krippendorffs_alpha(df_list, cols_to_check)
for col, alpha in alphas.items():
    print(f"Krippendorff's Alpha for {col}: {alpha:.3f}")



Skipping Evaluative: Only one unique value ([1]) after filtering for Evaluative=1.
Krippendorff's Alpha for Affect: 0.471
Krippendorff's Alpha for Judgement: 0.230
Krippendorff's Alpha for Appreciation: 0.330
Krippendorff's Alpha for Ambiguous: -0.071


In [ ]:

"""def save_agreement_disagreement_csvs_allcols(df, df2, columns_to_check, output_agree='agree.csv', output_disagree='disagree.csv'):

    Save two CSVs with all columns:
    - rows where annotators agree in ALL specified columns_to_check
    - rows where annotators disagree in ANY of those columns

    Parameters:
        df1, df2 (pd.DataFrame): Annotator DataFrames, aligned by index
        columns_to_check (list): Columns to check agreement on
        output_agree (str): Filename for agreed sentences CSV
        output_disagree (str): Filename for disagreed sentences CSV

    # Agreement mask for all specified columns
    agreement_mask = pd.Series(True, index=df.index)
    for col in columns_to_check:
        agreement_mask &= (df[col] == df2[col])

    # Select all columns from df1 (or df2, they should have same columns)
    agreed_rows = df.loc[agreement_mask].copy()
    disagreed_rows = df.loc[~agreement_mask].copy()

    agreed_rows.to_csv(output_agree, index=False)
    disagreed_rows.to_csv(output_disagree, index=False)

    print(f"Saved {len(agreed_rows)} agreed rows to {output_agree}")
    print(f"Saved {len(disagreed_rows)} disagreed rows to {output_disagree}")

cols_to_check = ['Affect', 'Judgement', 'Appreciation']

save_agreement_disagreement_csvs_allcols(df, df2, cols_to_check,
                                         output_agree='agreed_full_Bus11.csv',
                                         output_disagree='disagreed_full_Buss11.csv')"
"""


# mistral-large-instruct and Gold standard

Evaluative: Cohen's Kappa: 0.35751613894866274
Skipping Evaluative: Only one unique value ([1]) after filtering for Evaluative=1.

Krippendorff's Alpha for Affect: 0.367

Krippendorff's Alpha for Judgement: 0.326

Krippendorff's Alpha for Appreciation: 0.295

Krippendorff's Alpha for Ambiguous: -0.066

In [132]:

import pandas as pd
import numpy as np

#df = pd.read_excel("non-arg-phatic.xlsx")

df = pd.read_csv("/content/mistral-large-instruct.csv")
df2 = pd.read_excel("/content/Gold_standard_Mirela.xlsx")


In [133]:
df.columns

Index(['Unnamed: 0', 'Text_ID', 'Sentence_ID', 'Sentence', 'Evaluative',
       'Affect', 'Judgement', 'Appreciation', 'Ambiguous', 'binary',
       'binary_justification', 'binary_evidence_span', 'multiclass_label',
       'multiclass_probability_affect', 'multiclass_probability_judgment',
       'multiclass_probability_appreciation', 'multiclass_spans',
       'multiclass_explanation', 'binary_raw_output', 'multiclass_raw_output',
       'Source_File'],
      dtype='object')

In [134]:
df = df.drop(['Unnamed: 0', 'Sentence', 'Evaluative','multiclass_label',
       'Affect', 'Judgement', 'Appreciation', 'Ambiguous', 'Source_File','multiclass_explanation','binary_justification','binary_raw_output', 'multiclass_raw_output','multiclass_spans','multiclass_spans','binary_evidence_span'],axis = "columns")

In [135]:
# Columns to process
prob_cols = [
    "multiclass_probability_affect",
    "multiclass_probability_judgment",
    "multiclass_probability_appreciation"
]

# 1. Convert NaN (and string 'nan') to 0
df[prob_cols] = df[prob_cols].replace(["nan", "NaN", "NAN"], np.nan)
df[prob_cols] = df[prob_cols].fillna(0)

# 2. Convert to numeric and threshold >=0.5 → 1 else → 0
df[prob_cols] = df[prob_cols].apply(pd.to_numeric, errors="coerce").fillna(0)
df[prob_cols] = (df[prob_cols] >= 0.5).astype(int)

# 3. Create Ambiguous = 1 if more than 2 of the 3 values are 1
df["Ambiguous"] = (df[prob_cols].sum(axis=1) > 2).astype(int)

# 4. Rename columns
df = df.rename(columns={
    "multiclass_probability_affect": "Affect",
    "multiclass_probability_judgment": "Judgement",
    "multiclass_probability_appreciation": "Appreciation",
    "binary": "Evaluative"
})



In [136]:

# CHECK WHAT OTHER LABELS ARE TRUE PER ANNOTATOR IF EVALUATIVE = 0

df.columns

df = df.fillna(0)
#df2 = df2.drop(index=5)

df['Evaluative'] = df['Evaluative'].astype(int)
df['Affect'] = df['Affect'].astype(int)
df['Judgement'] = df['Judgement'].astype(int)
df['Appreciation'] = df['Appreciation'].astype(int)
df['Ambiguous'] = df['Ambiguous'].astype(int)
#df['Uncertain'] = df['Uncertain'].astype(int)


In [137]:
df2 = df2.fillna(0)

df2['Evaluative'] = df2['Evaluative'].astype(int)
df2['Affect'] = df2['Affect'].astype(int)
df2['Judgement'] = df2['Judgement'].astype(int)
df2['Appreciation'] = df2['Appreciation'].astype(int)
df2['Ambiguous'] = df2['Ambiguous'].astype(int)
#df2['Uncertain'] = df2['Uncertain'].astype(int)

In [138]:
df = df.drop_duplicates(subset=["Text_ID", "Sentence_ID"])
df = df.set_index(["Text_ID", "Sentence_ID"])
df2_idx = df2.set_index(["Text_ID", "Sentence_ID"]).index

df = df.loc[df2_idx].reset_index()
df

,Text_ID,Sentence_ID,Evaluative,Affect,Judgement,Appreciation,Ambiguous
0,Art5.,S6,0,0,0,0,0
1,Art5.,S120,1,1,0,0,0
2,Art5.,S51,1,1,0,0,0
3,Art5.,S131,1,1,0,0,0
4,Art5.,S130,1,1,0,0,0
...,...,...,...,...,...,...,...
435,Tech21.,S105,1,0,0,1,0
436,Tech21.,S30,0,0,0,0,0
437,Tech21.,S68,0,0,0,0,0
438,Tech21.,S66,0,0,0,0,0


In [139]:
df2

,Text_ID,Sentence_ID,Sentence,Evaluative,Affect,Judgement,Appreciation,Ambiguous
0,Art5.,S6,"Yes, eight kids in my family.",1,0,0,1,0
1,Art5.,S120,"When the idea hit me of how to do this, I got ...",1,1,0,1,1
2,Art5.,S51,"When she told me this, I fell to my knees.",1,1,0,0,0
3,Art5.,S131,"We're going to lose our health, people we love.",1,1,0,0,0
4,Art5.,S130,We're all going to have big losses in our live...,1,1,0,0,0
...,...,...,...,...,...,...,...,...
435,Tech21.,S105,We are less connected to it.,1,1,1,0,1
436,Tech21.,S30,Was the Internet actually a place that you cou...,0,0,0,0,0
437,Tech21.,S68,"Three years ago, when I started thinking about...",0,0,0,0,0
438,Tech21.,S66,"This is in Halifax, a cable that stretches fro...",0,0,0,0,0


## Analysis of Evaulative column = 0 and other columns = 1

In [140]:

df[(df['Evaluative'].isna()) | (df['Evaluative'] == "")]

df2[(df2['Evaluative'].isna()) | (df2['Evaluative'] == "")]

,Text_ID,Sentence_ID,Sentence,Evaluative,Affect,Judgement,Appreciation,Ambiguous


In [141]:
cols_annotator_1  = ['Evaluative', 'Affect', 'Judgement', 'Appreciation', 'Ambiguous']
cols_annotator_2  = ['Evaluative', 'Affect', 'Judgement', 'Appreciation', 'Ambiguous']

In [142]:
# Filter rows where Evaluative == 0 for annotator 1
df_0_eval_1 = df[df['Evaluative'] == 0]

In [143]:
# Count how many times each of the other columns == 1 in those rows
counts_1 = df_0_eval_1[cols_annotator_1].apply(lambda col: (col == 1).sum())
counts_1

,0
Evaluative,0
Affect,0
Judgement,0
Appreciation,0
Ambiguous,0


In [144]:
# Repeat for annotator 2
df_0_eval_2 = df[df['Evaluative'] == 0]
counts_2 = df_0_eval_2[cols_annotator_2].apply(lambda col: (col == 1).sum())

In [145]:
print("Annotator 1 - counts where Evaluative=0 and other columns=1:\n", counts_1)
print("Annotator 2 - counts where Evaluative=0 and other columns=1:\n", counts_2)

Annotator 1 - counts where Evaluative=0 and other columns=1:
 Evaluative      0
Affect          0
Judgement       0
Appreciation    0
Ambiguous       0
dtype: int64
Annotator 2 - counts where Evaluative=0 and other columns=1:
 Evaluative      0
Affect          0
Judgement       0
Appreciation    0
Ambiguous       0
dtype: int64


## Analysis of Evaulative column = 1

In [146]:
# Filter rows where Evaluative == 1 for annotator 1
df_0_eval_1 = df[df['Evaluative'] == 1]

In [147]:
# Count how many times each of the other columns == 1 in those rows
counts_1 = df_0_eval_1[cols_annotator_1].apply(lambda col: (col == 1).sum())
counts_1

,0
Evaluative,144
Affect,29
Judgement,28
Appreciation,87
Ambiguous,0


In [148]:
# Repeat for annotator 2
df_0_eval_2 = df2[df2['Evaluative'] == 1]
counts_2 = df_0_eval_2[cols_annotator_2].apply(lambda col: (col == 1).sum())
counts_2

,0
Evaluative,268
Affect,61
Judgement,127
Appreciation,107
Ambiguous,27


In [149]:
print("Annotator 1 - counts where Evaluative=1 and other columns=1:\n", counts_1)
print("Annotator 2 - counts where Evaluative=1 and other columns=1:\n", counts_2)

Annotator 1 - counts where Evaluative=1 and other columns=1:
 Evaluative      144
Affect           29
Judgement        28
Appreciation     87
Ambiguous         0
dtype: int64
Annotator 2 - counts where Evaluative=1 and other columns=1:
 Evaluative      268
Affect           61
Judgement       127
Appreciation    107
Ambiguous        27
dtype: int64


## Berechnung: Cohens_Kappa

In [150]:
!pip install krippendorff

In [151]:
from sklearn.metrics import cohen_kappa_score


In [152]:
"""# Filter out the placeholder values before calculating the kappa score
df_anno1_filtered = df_anno1[df_anno1["Evaluative"] != -1]
df2_anno2_filtered = df2_anno2[df2_anno2["Evaluative"] != -1]

# Ensure both dataframes have the same indices after filtering
common_indices = df_anno1_filtered.index.intersection(df2_anno2_filtered.index)
df_anno1_aligned = df_anno1_filtered.loc[common_indices]
df2_anno2_aligned = df2_anno2_filtered.loc[common_indices]"""

'# Filter out the placeholder values before calculating the kappa score\ndf_anno1_filtered = df_anno1[df_anno1["Evaluative"] != -1]\ndf2_anno2_filtered = df2_anno2[df2_anno2["Evaluative"] != -1]\n\n# Ensure both dataframes have the same indices after filtering\ncommon_indices = df_anno1_filtered.index.intersection(df2_anno2_filtered.index)\ndf_anno1_aligned = df_anno1_filtered.loc[common_indices]\ndf2_anno2_aligned = df2_anno2_filtered.loc[common_indices]'

In [153]:
# Assuming your columns are named "mobilisierung" and "informativ" (adjust as necessary)
kappa_score = cohen_kappa_score(df["Evaluative"], df2["Evaluative"])
print(f"Cohen's Kappa: {kappa_score}")

Cohen's Kappa: 0.35751613894866274


In [154]:
#Kappa Kuck und Knierim: Cohen's Kappa: 0.6156981445684476
# Kappa Knierim und GPT4: Cohen's Kappa: 0.35816326807273824
# Kappa Kuck und GPT4: Cohen's Kappa: 0.38653552825615034

In [155]:
#filter the empty string-lines to calculate agreement on whether the sentence is evaluative
"""df[df['Evaluative'] == 1]
df2[df2['Evaluative'] == 1]
"""

"df[df['Evaluative'] == 1]\ndf2[df2['Evaluative'] == 1]\n"

In [156]:
import krippendorff
import numpy as np

def calculate_krippendorffs_alpha(df_list, columns, evaluative_col='Evaluative'):
    """
    Calculates Krippendorff's Alpha for given columns, considering only rows
    where evaluative_col == 1. Assumes missing data is marked as -1.

    Parameters:
        df_list (list of pd.DataFrame): List of annotator DataFrames (one per annotator).
        columns (list): List of column names to calculate alpha on.
        evaluative_col (str): Column name indicating 'true' evaluative lines.

    Returns:
        dict: column_name -> Krippendorff's Alpha score
    """
    alphas = {}

    # Combine DataFrames on index (assumes they are aligned)
    # Create a matrix of shape (num_annotators, num_items)
    for col in columns:
        # Filter rows where evaluative_col == 1 for all annotators
        mask = np.array([df[evaluative_col] == 1 for df in df_list]).all(axis=0)

        # Collect data from all annotators for this column, filtered by mask
        data = []
        for df in df_list:
            # Use -1 as missing, krippendorff expects np.nan for missing, so convert
            col_data = df.loc[mask, col].replace(-1, np.nan).to_numpy()
            data.append(col_data)

        # Convert to numpy array of shape (num_annotators, num_items)
        data = np.array(data)

        # Check if there is more than one unique value in the data for this column
        unique_values = np.unique(data[~np.isnan(data)])
        if len(unique_values) <= 1:
            print(f"Skipping {col}: Only one unique value ({unique_values}) after filtering for Evaluative=1.")
            continue

        # Calculate Krippendorff's Alpha (nominal metric)
        alpha = krippendorff.alpha(reliability_data=data, level_of_measurement='nominal')
        alphas[col] = alpha

    return alphas

df_list = [df, df2]
cols_to_check = ['Evaluative','Affect', 'Judgement', 'Appreciation', 'Ambiguous']

alphas = calculate_krippendorffs_alpha(df_list, cols_to_check)
for col, alpha in alphas.items():
    print(f"Krippendorff's Alpha for {col}: {alpha:.3f}")



Skipping Evaluative: Only one unique value ([1]) after filtering for Evaluative=1.
Krippendorff's Alpha for Affect: 0.367
Krippendorff's Alpha for Judgement: 0.326
Krippendorff's Alpha for Appreciation: 0.295
Krippendorff's Alpha for Ambiguous: -0.066


In [157]:

"""def save_agreement_disagreement_csvs_allcols(df, df2, columns_to_check, output_agree='agree.csv', output_disagree='disagree.csv'):

    Save two CSVs with all columns:
    - rows where annotators agree in ALL specified columns_to_check
    - rows where annotators disagree in ANY of those columns

    Parameters:
        df1, df2 (pd.DataFrame): Annotator DataFrames, aligned by index
        columns_to_check (list): Columns to check agreement on
        output_agree (str): Filename for agreed sentences CSV
        output_disagree (str): Filename for disagreed sentences CSV

    # Agreement mask for all specified columns
    agreement_mask = pd.Series(True, index=df.index)
    for col in columns_to_check:
        agreement_mask &= (df[col] == df2[col])

    # Select all columns from df1 (or df2, they should have same columns)
    agreed_rows = df.loc[agreement_mask].copy()
    disagreed_rows = df.loc[~agreement_mask].copy()

    agreed_rows.to_csv(output_agree, index=False)
    disagreed_rows.to_csv(output_disagree, index=False)

    print(f"Saved {len(agreed_rows)} agreed rows to {output_agree}")
    print(f"Saved {len(disagreed_rows)} disagreed rows to {output_disagree}")

cols_to_check = ['Affect', 'Judgement', 'Appreciation']

save_agreement_disagreement_csvs_allcols(df, df2, cols_to_check,
                                         output_agree='agreed_full_Bus11.csv',
                                         output_disagree='disagreed_full_Buss11.csv')"
"""


'def save_agreement_disagreement_csvs_allcols(df, df2, columns_to_check, output_agree=\'agree.csv\', output_disagree=\'disagree.csv\'):\n\n    Save two CSVs with all columns:\n    - rows where annotators agree in ALL specified columns_to_check\n    - rows where annotators disagree in ANY of those columns\n\n    Parameters:\n        df1, df2 (pd.DataFrame): Annotator DataFrames, aligned by index\n        columns_to_check (list): Columns to check agreement on\n        output_agree (str): Filename for agreed sentences CSV\n        output_disagree (str): Filename for disagreed sentences CSV\n\n    # Agreement mask for all specified columns\n    agreement_mask = pd.Series(True, index=df.index)\n    for col in columns_to_check:\n        agreement_mask &= (df[col] == df2[col])\n\n    # Select all columns from df1 (or df2, they should have same columns)\n    agreed_rows = df.loc[agreement_mask].copy()\n    disagreed_rows = df.loc[~agreement_mask].copy()\n\n    agreed_rows.to_csv(output_agree, 